In [1]:
import tensorflow as tf
import math
import numpy as np
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import itertools
from nltk.tokenize import RegexpTokenizer
from functools import partial

In [4]:
# conv.txt
tokenizer = RegexpTokenizer(r'\w+')
def process_sentence(x, eos=False):
#     x = x.split()
    x = tokenizer.tokenize(x)
    x = [i.lower() for i in x]
    if eos:
        x = ['eos'] + x + ['eos']
    return x

def one_hot(x):
    t = [0] * vocab_size
    t[x] = 1
    return t
    
def sent_encode(x):
    x = [w_i[i] for i in x]
    x = map(one_hot, x)
    return list(x)

data=pd.read_csv('conv.txt', delimiter='\t', error_bad_lines=False, header=None)
data.dropna(inplace=True)
inp = list(map(partial(process_sentence), data[0].values))
out = list(map(partial(process_sentence, eos=True), data[1].values))
words=np.r_[inp, out]
words = list(set(itertools.chain(*words)))
vocab_size=len(words)
i_w = dict(enumerate(words))
w_i = {v:k for k, v in i_w.items()}
inp = np.array(list(map(sent_encode, inp)))
out = np.array(list(map(sent_encode, out)))
print(vocab_size, inp.shape, out.shape)

b'Skipping line 1681: expected 2 fields, saw 3\nSkipping line 1765: expected 2 fields, saw 3\nSkipping line 1984: expected 2 fields, saw 3\nSkipping line 2046: expected 2 fields, saw 3\nSkipping line 2086: expected 2 fields, saw 3\nSkipping line 2313: expected 2 fields, saw 3\nSkipping line 2314: expected 2 fields, saw 3\n'


3325 (2330,) (2330,)


In [5]:
batch_size = 5
input_size = len(inp[0][0])
dim = 256
input_size, dim

(3325, 256)

In [6]:
max_length1 = max([len(x) for x in inp])
max_length2 = max([len(x) for x in out])
print(max_length1, max_length2)
def get_batch(size):
    idx = np.random.choice(range(len(inp)), size=size, replace=False)
    x, y, y_ = inp[idx], list(map(lambda x: x[:-1], out[idx])), list(map(lambda x: x[1:], out[idx]))
    x = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length1 - len(z))], x))
    y = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length2 - len(z))], y))
    y_ = list(map(lambda z: [i for i in z] +
                 [[0] * input_size for i in range(max_length2 - len(z))], y_))
    x, y, y_ = np.array(x), np.array(y), np.array(y_)
    return x, y, y_
a, b, c = get_batch(5)
# print(np.argmax(b[0], axis=1))
# print(np.argmax(c[0], axis=1))

70 139


In [7]:
tf.reset_default_graph()
with tf.Graph().as_default() as graph:
    def length(sequence):
        ''' returns length of sequences in a batch '''
        used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length
    
    def cost(output, target):
        cross_entropy = target * tf.log(output)
        cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
        mask = tf.sign(tf.reduce_max(tf.abs(target), reduction_indices=2))
        cross_entropy *= mask
        cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.reduce_sum(mask, reduction_indices=1)
        return tf.reduce_mean(cross_entropy)
    
    def effective_sequence(sequences, length, dim):
        ''' returns sequences after all end zeros are stripped '''
        n = len(sequences)
        return [tf.slice(sequences[i], [0, 0], [tf.nn.embedding_lookup(length, i), dim]) for i in range(n)]
    
    def display_seq(seq):
        def seq_id_to_word(x):
            return [i_w[i] for i in x]
        print(np.argmax(seq, axis=2).shape)
        temp=list(map(np.trim_zeros, np.argmax(seq, axis=2)))
        temp = list(map(seq_id_to_word, temp))
        for t in temp:
            print(' '.join(t))
    
    with tf.variable_scope('encoder'):
        enc_seq = tf.placeholder(tf.float32, [batch_size, max_length1, input_size])
        enc_seq_length = length(enc_seq)
        enc_out, enc_final = tf.nn.dynamic_rnn(tf.nn.rnn_cell.GRUCell(dim), enc_seq, 
                                      sequence_length=enc_seq_length, dtype=tf.float32)
        
    with tf.variable_scope('decoder'):    
        dec_tar = tf.placeholder(shape=[batch_size, max_length2, input_size], dtype=tf.float32)
        dec_seq = tf.placeholder(tf.float32, [batch_size, max_length2, input_size])
        dec_seq_length = length(dec_seq)
        dec_out = tf.nn.dynamic_rnn(tf.nn.rnn_cell.GRUCell(dim), 
                                        dec_seq, sequence_length=dec_seq_length, initial_state=enc_final)[0]
        
#         dec_out = list(map(tf.squeeze, tf.split(0, batch_size, dec_out)))
#         dec_out = effective_sequence(dec_out, dec_seq_length, dim)
        
    Wy = tf.Variable(tf.random_uniform(shape=[dim, input_size],
                                      minval=-1, maxval=1), dtype=tf.float32)
    by = tf.Variable(tf.zeros(shape=[input_size]))
    yp = tf.nn.softmax(tf.einsum('ijk,kl->ijl', dec_out, Wy) + by)
    loss = cost(yp, dec_tar)
    optimize_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

    saver = tf.train.Saver()
    with tf.Session(graph=graph) as sess:
        sess.run(tf.initialize_all_variables())
#         new_saver = tf.train.import_meta_graph('model.ckpt.meta')
#         new_saver.restore(sess, tf.train.latest_checkpoint('./'))
        avg_loss = 0
        for i in range(1000):
            a, b, c = get_batch(batch_size)
            _, l = sess.run([optimize_op, loss], {enc_seq: a, dec_seq: b, dec_tar: c})
            avg_loss += l
            break
            if i % 50 == 0 and i != 0:
                print(avg_loss / 50)
                avg_loss = 0
                saver.save(sess, 'model.ckpt')

In [ ]:
input_size = vocab_size
print(input_size)
dims = [300, 200, 300]
num_layers = 3

In [ ]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxz']) + tf.matmul(p, enc_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, enc_layers[j]['Wxr']) + tf.matmul(p, enc_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, enc_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), enc_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

def recurrence2(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxz']) + tf.matmul(p, dec_layers[j]['Whz']))
    r = tf.nn.sigmoid(tf.matmul(i, dec_layers[j]['Wxr']) + tf.matmul(p, dec_layers[j]['Whr']))
    h_ = tf.nn.tanh(tf.matmul(i, dec_layers[j]['Wxh']) +
                    tf.matmul(tf.mul(p, r), dec_layers[j]['Whh']))
    h = tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p)
    return tf.reshape(h, [dims[j-1]])

In [ ]:
tf.reset_default_graph()
with tf.device('/cpu:0'):
    with tf.name_scope('encoder'):
        enc_layers = []
        incoming = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='encoder_incoming')
        enc_layers.append({'states': incoming, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [enc_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            layer['initial'] = tf.placeholder(shape=[dims[j-1]], dtype=tf.float32, name='initial'+str(j))
            enc_layers.append(layer)
            layer['states'] = tf.scan(recurrence, enc_layers[-2]['states'], initializer=layer['initial'], name='encoder_states'+str(j))
            layer['last'] = tf.squeeze(tf.expand_dims(tf.nn.embedding_lookup(layer['states'], 
                                                                             tf.shape(layer['states'])[0] - 1), dim=0),
                                      name='last'+str(j))

In [ ]:
with tf.device('/cpu:0'):
    with tf.name_scope('decoder'):
        
        dec_layers = []
        decoder_input = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoeder_input')
        dec_layers.append({'states': decoder_input, 'dim': input_size})
        
        for j in range(1, num_layers + 1):
            shape = [dec_layers[-1]['dim'], dims[j-1]]
            layer = dict()
            layer['Wxh'] = tf.Variable(tf.random_normal(
                shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whh'] = tf.Variable(tf.random_normal(
                [dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            
        # layer associated with update gate
            layer['Wxz'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whz'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]], stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
        # layer associated with the reset gate
            layer['Wxr'] = tf.Variable(tf.random_normal(shape,
                stddev=1.0 / math.sqrt(shape[0])), dtype=tf.float32)
            layer['Whr'] = tf.Variable(tf.random_normal(shape=[
                dims[j-1], dims[j-1]],
                stddev=1.0 / math.sqrt(dims[j-1])), dtype=tf.float32)
            layer['dim'] = dims[j-1]
            dec_layers.append(layer)
            
            layer['states'] = tf.scan(recurrence2, dec_layers[-2]['states'], initializer=enc_layers[j]['last'], 
                                      name='decoder_states'+str(j))
            

        # softmax layer
        Why = tf.Variable(tf.random_normal(
            [dims[-1], input_size],
            stddev=1.0 / math.sqrt(dims[-1])), dtype=tf.float32)
        by = tf.Variable(tf.zeros(input_size), dtype=tf.float32)
        outputs = tf.matmul(dec_layers[-1]['states'], Why) + by
        
        # decoding step (with 'teacher forcing')

        decoder_target = tf.placeholder(shape=[None, input_size], dtype=tf.float32, name='decoder_target')
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(outputs, decoder_target))
        optimizer_op = tf.train.AdagradOptimizer(0.1).minimize(loss)

In [ ]:
saver = tf.train.Saver()
sess =  tf.Session()
saver.restore(sess, 'model.ckpt')
# sess.run(tf.initialize_all_variables())
ix = 0
avg_loss = 0

In [ ]:
# beam search output generator (explained here: http://goo.gl/g3jjYs)

def beam_generate(inp, size=2):
    inp = process_sentence(inp)
    inp = sent_encode(inp)
    out = ['eos']
    seed = [   [one_hot(w_i['eos'])]   ]
    temp = []
    for j in range(10):
        for s in seed:
            feed = {k:v for k,v in zip([l['initial'] for l in enc_layers[1:]], [np.zeros(d) for d in dims])}
            feed.update({enc_layers[0]['states'] : inp, dec_layers[0]['states']: s})        
            o = sess.run(outputs, feed_dict=feed)[-1]
            r = np.argsort(o)[-size:]
            t = list(np.argmax(s, axis=1))
#             print('r', r)
#             print('t', t)
            for i in r:
#                 print('t+i', t + [i])
                temp.append(s + [one_hot(i)])
#         print(len(temp))
        temp = temp[:size]
        seed = temp
        temp = []
    seed=[np.argmax(s, axis=1) for s in seed]
    for s in seed:
        t=''
        for j in s:
            t+=' '+i_w[j]
        print(t)
        
            
def generate_out(inp):       
    inp = process_sentence(inp)
    inp = sent_encode(inp)
    out = ['eos']
    seed = [one_hot(w_i['eos'])]
    while len(seed) < 20:
        feed = {k:v for k,v in zip([l['initial'] for l in enc_layers[1:]], [np.zeros(d) for d in dims])}
        feed.update({enc_layers[0]['states'] : inp, dec_layers[0]['states']: seed})
        o = sess.run(outputs, feed_dict=feed)[-1]
        r = i_w[np.argmax(o)]
        seed.append(o)
        out.append(r)
        if r == 'eos':
            break
    print(' '.join(out[1:-1]))

In [ ]:
print_step = 1000
for i in range(2 * len(inp)):
    ix = np.random.randint(0, len(inp))
    a = inp[ix]
    b = out[ix]
    feed = {incoming: a, decoder_target: b[1:], decoder_input : b[:-1]}
    for j in enc_layers[1:]:
        feed[j['initial']] = np.zeros(j['dim'])
        
    l, _, o = sess.run([loss, optimizer_op, outputs], feed_dict=feed)
    avg_loss += l
    if i % print_step == 0 and i != 0:
        print(avg_loss / print_step)
        avg_loss = 0
        saver.save(sess, 'checkpoint.ckpt')
        generate_out('do you have girlfriend')
        
    ix += 1

In [ ]:
# saver = tf.train.Saver()
saver.save(sess, 'model.ckpt')

In [ ]:
generate_out('thanks')

In [ ]:
data

In [ ]:
[i_w[x] for x in np.argmax(inp[0], axis=1)]